# Assignment 1-2: Data Collection Using Web APIs

## Objective

Many Websites (such as Twitter, Yelp, Spotify) provide free APIs to allow users to access their data. *API wrappers* simplify the use of these APIs by wrapping API calls into easy-to-use Python functions. At SFU, we are developing a unified API wrapper, called [DataPrep.Connector](https://docs.dataprep.ai/user_guide/connector/introduction.html#userguide-connector), which offers a unified programming interface to collect data from a variety of Web APIs.

In this assignment, you will learn the following:

* How to ask insightful questions about data
* How to collect data from Web APIs using DataPrep.Connector

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please follow the python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

DataPrep.Connector is very easy to learn. By watching this 10-min [PyData Global 2020 talk](https://www.youtube.com/watch?v=56qu-0Ka-dA), you should be able to know how to use it. 

If you want to know more, below are some other useful resources.

* [Quick Introduction](https://github.com/sfu-db/dataprep#connector)
* [User Guide](https://sfu-db.github.io/dataprep/user_guide/connector/connector.html) 
* [Examples](https://github.com/sfu-db/dataprep/tree/develop/examples)
* [Fetch and analyze COVID-19 tweets using DataPrep](https://www.youtube.com/watch?v=vvypQB3Vp1o)

## Overview

This is a **group** assignment. 
Please check your group in this [PDF file](https://coursys.sfu.ca/2022sp-cmpt-733-g1/pages/Web-API-Assignment/view).

To do this assignment, your group needs to go through four steps:

1. Select a new Web API that is not listed on https://github.com/sfu-db/APIConnectors. 
2. Create a configuration file for the API (see tutorials at [link1](https://github.com/sfu-db/APIConnectors/blob/develop/CONTRIBUTING.md#add-configuration-files) and [link2](https://github.com/sfu-db/EZHacks-tutorial/blob/master/2.%20Tutorial.ipynb)). 
3. Come up with four questions about the API. 
4. Write code to answer these questions one by one.

For Step 3, please make sure your questions are **good**.

## What are "good questions"?

Please use the following to judge whether your questions are good or not.

1. Good questions need to be useful. That is, they are either common questions asked about the API or are exploring novel use cases.
2. Good questions need to be diverse. That is, they cover different aspects of the API. 
3. Good questions have to cover different difficulty levels. That is, it consists of both easy and difficult questions,  where the difficulty can be measured by the number of lines of code or the number of input parameters.

The following shows four good questions about the Yelp API. The corresponding code can be found at this [link](https://github.com/sfu-db/DataConnectorConfigs#yelp----collect-local-business-data).

* Q1. What's the phone number of Capilano Suspension Bridge Park?
* Q2. Which yoga store has the highest review count in Vancouver?
* Q3. How many Starbucks stores are in Seattle and where are they?
* Q4. What are the ratings for a list of restaurants?

**Why are they useful?**
* Q1 is useful because "Capilano Suspension Bridge Park" is one of the most popular tourist attractions in Vancouver.
* Q2 is useful because a yoga fan wants to find out the most popular yoga store in Vancouver. 
* Q3 is useful because Starbucks was founded in Seattle.
* Q4 is useful because people often rely on yelp ratings to decide which restaurant to go to.

**Why are they diverse?**

This is because the [code](yelp-code.png) written to answer them has different inputs or outputs.
* Q1 takes `term` and `location` as input and returns 1 record with attributes `name` and `phone` 
* Q2 takes `categories`, `location`, and `sort_by` as input and returns 1 record with attributes `name` and `review_count`
* Q3 takes `term` and `location` as input and returns n records with attributes `name`, `address`, `city`, `state`, `country`, `zipcode`
* Q4 takes a list of retarurant `names` as input and return n records with attributes `name`, `rating`, `city`

**Why are they more and more difficult?**
* Q1 (4 lines of code, 2 query parameters)
* Q2 (4 lines of code, 3 query parameters)
* Q3 (5 lines of code, 2 query parameters)
* Q4 (11 lines of code, 2 query parameters)

Please note that you have to use DataPrep.Connector to get data from the Web API. If DataPrep.Connector cannot meet your needs, please post your questions on Teams (Channel: Assignment 1). We will help you. 

## Now, it's your turn. :) 

Please write down your questions and the corresponding code for your assigned API. 

In [1]:
## Provide your API key here for TAs to reproduce your results
access_token = 'UFTrxukJoVW0mAEUyGwelA))'
key = "ya6VKb)ixz5rGXSM1vnCWA(("

In [2]:
# Connecting to DataPrep
from mmap import PAGESIZE
from dataprep.connector import connect
from time import sleep
import datetime
import pandas as pd
from datetime import datetime

dc = connect('./config', _concurrency=10)

### Q1

What are 3 most popular topics in stackoverflow?

In [3]:
tags_df = await dc.query('tags', _count=3, _auth={'access_token': access_token}, site='stackoverflow', key=key, sort='popular')
tags_df

,name
0,javascript
1,python
2,java


### Q2

What percentage of the questions were not answered on Stackoverflow with the latest revision?

In [4]:
df = await dc.query('info', term = 'api_revision', _auth={'access_token': access_token}, key=key, site='stackoverflow')
df['percentage'] = df['total_unanswered']/df['total_questions']*100
df[['percentage','api_revision']]

,percentage,api_revision
0,30.604313,2023.1.13.36624


### Q3

Who are the top 3 users with highest reputation among all the users and what are their badge counts?

In [5]:
df_most_reputation = await dc.query('users', _count=3 , _auth={'access_token': access_token}, site='stackoverflow', key=key, 
                     sort='reputation', order='desc')
df_most_reputation['creation_date'] = df_most_reputation['creation_date'].apply(lambda x: datetime.fromtimestamp(x).strftime("%m/%d/%Y, %H:%M:%S"))

df_most_reputation = df_most_reputation.drop(['badge_counts'], axis = 1)
df_most_reputation['total_badge_count'] = df_most_reputation['gold'] + df_most_reputation['silver'] + df_most_reputation['bronze']
df_most_reputation

,gold,silver,bronze,account_id,location,display_name,reputation,creation_date,total_badge_count
0,848,9033,9129,11683,"Reading, United Kingdom",Jon Skeet,1380049,"09/26/2008, 05:05:05",19010
1,54,619,763,1165580,"New York, United States",Gordon Linoff,1223134,"01/11/2012, 11:53:57",1436
2,505,4233,5042,4243,France,VonC,1182910,"09/13/2008, 15:22:33",9780


### Q4

How many users from Canada joined StackExchange in the month of December, 2022 and what is the average reputation for the new users until now?

In [6]:
start_date_epoch = datetime(2022, 12, 1, 0, 0, 0).strftime('%s')
end_date_epoch = datetime(2022, 12, 31, 0, 0, 0).strftime('%s')

df = await dc.query('users', _count=1000 , _auth={'access_token': access_token}, site='stackoverflow', key=key, fromdate = start_date_epoch, todate = end_date_epoch)

df = df.drop(['badge_counts','gold','silver','bronze'], axis = 1)
df['creation_date'] = df['creation_date'].apply(lambda x: datetime.fromtimestamp(x).strftime("%m/%d/%Y, %H:%M:%S"))
df = df.dropna()
mask = df[df['location'].str.contains("Canada")]
print("Average Reputation - ",mask['reputation'].mean())
mask

Average Reputation -  404.5


,account_id,location,display_name,reputation,creation_date
0,11231367,"Montr&#233;al, QC, Canada",Ianatore,111,"12/13/2022, 07:47:45"
354,27370501,"Vancouver, BC. Canada",ricardkelly,698,"12/27/2022, 20:44:51"


### Q5

Which topic was most discussed on 15th Jan 2023 and what was the question?

In [7]:
start_date_epoch = datetime(2023, 1, 15, 0, 0, 0).strftime('%s')
end_date_epoch = datetime(2023, 1, 16, 0, 0, 0).strftime('%s')

comments_df = await dc.query('comments', _count=10000, _auth={'access_token': access_token}, site='stackoverflow', key=key, sort='votes', fromdate=start_date_epoch, todate=end_date_epoch)

aggregated_comments_df = comments_df.groupby(['post_id'])['post_id'].count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False)

question_df = await dc.query('search', _count=1, _auth={'access_token': access_token}, q='inquestion:'+str(aggregated_comments_df.iloc[0]['post_id']) ,site='stackoverflow', key=key, order ='desc', sort='votes')
tags = ', '.join(question_df.iloc[0][ "tags"])
title = question_df.iloc[0][ "title"]
print("The most discussed topics on 15th Jan 2023 are ["+tags+ "] and the question was {"+title+"}")

The most discussed topics on 15th Jan 2023 are [reactjs, react-three-fiber] and the question was {Models in React Three Fiber flicker and glitch, but work fine in Codesandbox and also work fine in pure Three.js}


## Submission

Complete this notebook, rename it to `A1-2-[WEB API Name].ipynb`, and submit it along with your config files to the CourSys activity `Assignment 1 - Part 2`. For example, if your group works on Yelp, then **every member of your group** needs to submit the same notebook named `A1-2-Yelp.ipynb` and the config files named `config.zip`.